# Board Game Behavior: An Analysis of Demographics and Gameplay Preferences

## Documentation

[Initial Project Proposal]("write-ups/initial_project_proposal.md")

## Data Processing and Modeling

### Data Cleaning

### Exploratory Data Analysis

### Machine Learning